In [1]:
import os
import re
import sys
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

cmap_name = 'viridis'
verbose = True

In [2]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

Successfully obtained PowerFactory application.


In [3]:
project_name = '\\Terna_Inerzia\\IEEE39_rnd_load'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

Successfully activated project \Terna_Inerzia\IEEE39_rnd_load.


In [4]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

There are 10 generators.
There are 34 lines.
There are 39 buses.
There are 19 loads.


In [24]:
line_buses = lambda line: (int(re.findall('\d+', line.bus1.cterm.loc_name)[0]), \
                           int(re.findall('\d+', line.bus2.cterm.loc_name)[0]))
def line_buses_str(line):
    a,b = line_buses(line)
    return f'{a:02d}_{b:02d}'

In [31]:
lengths = {line_buses_str(line): line.dline for line in lines}
R = {line_buses_str(line): line.typ_id.rline for line in lines}
X = {line_buses_str(line): line.typ_id.xline for line in lines}
keys = sorted(list(R.keys()))
lengths = {key: lengths[key] for key in keys}
R = {key: R[key] for key in keys}
X = {key: X[key] for key in keys}
W = {key: np.sqrt((R[key]*lengths[key])**2 + (X[key]*lengths[key])**2) for key in keys}

In [37]:
with open('IEEE39_graph.txt', 'w') as fid:
    for key,weight in W.items():
        bus1,bus2 = list(map(int, key.split('_')))
        fid.write(f'{bus1} {bus2} {weight}\n')